In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
import warnings
warnings.filterwarnings('ignore')

2024-05-21 13:07:31.971994: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 13:07:31.972093: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 13:07:32.104470: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def create_data_generator(data_dir, batch_size):
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        zoom_range=0.1,
        brightness_range=[0.5, 1.5],
        horizontal_flip=True,
        fill_mode='nearest'
    )
    generator = datagen.flow_from_directory(
        directory=data_dir,
        target_size=(224, 224),  # Resize images to 224x224 to match the input size of the model
        batch_size=batch_size,
        class_mode='binary'  # Binary labels
    )
    return generator

# Create generators
batch_size = 512
train_generator = create_data_generator('/kaggle/input/morph-splitted/train', batch_size)
val_generator = create_data_generator('/kaggle/input/morph-splitted/val', batch_size)
test_generator = create_data_generator('/kaggle/input/morph-splitted/test', batch_size)


Found 24000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.


### **ResNet50**

In [3]:
# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze layers
for layer in base_model.layers:
    layer.trainable = False

# Adding custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model_ResNet50 = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model_ResNet50.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [4]:
# Fit the model
batch_size = 512
epochs = 10
history = model_ResNet50.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)

Epoch 1/10
  2/750 ━━━━━━━━━━━━━━━━━━━━ 40s 54ms/step - accuracy: 0.6172 - loss: 0.6581   

I0000 00:00:1716296918.543616     126 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1716296918.593948     126 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 660ms/step - accuracy: 0.6439 - loss: 0.5987

W0000 00:00:1716297418.384009     125 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


750/750 ━━━━━━━━━━━━━━━━━━━━ 681s 884ms/step - accuracy: 0.6439 - loss: 0.5986 - val_accuracy: 0.6659 - val_loss: 0.5812
Epoch 2/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 58us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 546s 723ms/step - accuracy: 0.6973 - loss: 0.5333 - val_accuracy: 0.7212 - val_loss: 0.5097
Epoch 4/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 26us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 568s 753ms/step - accuracy: 0.7329 - loss: 0.4989 - val_accuracy: 0.7725 - val_loss: 0.4544
Epoch 6/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 31us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 579s 767ms/step - accuracy: 0.7622 - loss: 0.4597 - val_accuracy: 0.6679 - val_loss: 0.6017
Epoch 8/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 26us/

In [5]:
# Evaluate the model on test data
test_loss, test_accuracy = model_ResNet50.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy}")


250/250 ━━━━━━━━━━━━━━━━━━━━ 225s 902ms/step - accuracy: 0.8453 - loss: 0.3957
Test accuracy: 0.8418750166893005


In [6]:
fm_generator = create_data_generator('/kaggle/input/mad-benchmark/FaceMorpher', batch_size)
mg1_generator = create_data_generator('/kaggle/input/mad-benchmark/MIPGAN_I', batch_size)
mg2_generator = create_data_generator('/kaggle/input/mad-benchmark/MIPGAN_II', batch_size)
oc_generator = create_data_generator('/kaggle/input/mad-benchmark/OpenCV', batch_size)
wm_generator = create_data_generator('/kaggle/input/mad-benchmark/Webmorph', batch_size)

Found 1204 images belonging to 2 classes.
Found 1204 images belonging to 2 classes.
Found 1203 images belonging to 2 classes.
Found 1188 images belonging to 2 classes.
Found 704 images belonging to 2 classes.


In [7]:
datasets = [fm_generator, mg1_generator, mg2_generator, oc_generator,wm_generator]
results = []
losses = []
for i in datasets:
    test_loss, test_accuracy = model_ResNet50.evaluate(i)
    results.append(test_accuracy)
    losses.append(test_loss)

3/3 ━━━━━━━━━━━━━━━━━━━━ 60s 9s/step - accuracy: 0.4587 - loss: 1.3323 


W0000 00:00:1716300141.492812     126 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


3/3 ━━━━━━━━━━━━━━━━━━━━ 62s 18s/step - accuracy: 0.3530 - loss: 1.5515
3/3 ━━━━━━━━━━━━━━━━━━━━ 76s 25s/step - accuracy: 0.3767 - loss: 1.4309


W0000 00:00:1716300419.414279     125 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


3/3 ━━━━━━━━━━━━━━━━━━━━ 43s 15s/step - accuracy: 0.2664 - loss: 2.0968


W0000 00:00:1716300495.862246     124 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


2/2 ━━━━━━━━━━━━━━━━━━━━ 33s 19s/step - accuracy: 0.3209 - loss: 2.1240


W0000 00:00:1716300555.664577     125 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [10]:
names = ["FaceMorpher", "MIPGAN_I", "MIPGAN_II", "OpenCV", "Webmorph"]
for i,j in zip(names, results):
    print(i, ": ", j)

FaceMorpher :  0.45847177505493164
MIPGAN_I :  0.35548171401023865
MIPGAN_II :  0.36907729506492615
OpenCV :  0.2651515007019043
Webmorph :  0.33096590638160706


In [11]:
# from tensorflow.keras.models import load_model
# model.save(os.path.join('models','model_ResNet50.h5'))


In [12]:
# new_model = load_model('models/model_ResNet50.h5')

### **apcer_at_fixed_bpcer**

In [14]:
def calculate_apcer_at_fixed_bpcer(fpr, tpr, thresholds, fixed_bpcer):
    """Calculate the APCER at a fixed BPCER."""
    fpr_target = fixed_bpcer
    closest_fpr_index = np.argmin(np.abs(fpr - fpr_target))
    corresponding_apcer = 1 - tpr[closest_fpr_index]
    corresponding_threshold = thresholds[closest_fpr_index]
    return corresponding_apcer, corresponding_threshold

In [15]:
datasets = [fm_generator, mg1_generator, mg2_generator, oc_generator, wm_generator]
names = ["FaceMorpher", "MIPGAN_I", "MIPGAN_II", "OpenCV", "Webmorph"]
fixed_bpcer_values = [0.01, 0.1, 0.2]  # Define fixed BPCER values
all_results = []

# Iterate over each dataset
for dataset, name in zip(datasets, names):
    # Evaluate the model and get results
    index = datasets.index(dataset)  # Index of the current dataset
    test_loss = losses[index]
    test_accuracy = results[index]
    
    # Predictions and true labels
    predictions = model_ResNet50.predict(dataset)
    true_labels = dataset.classes
    if predictions.ndim > 1 and predictions.shape[1] > 1:
        predictions = predictions[:, 1]

    # ROC curve metrics
    fpr, tpr, thresholds = roc_curve(true_labels, predictions, pos_label=1)
    
    # Calculate APCER for each BPCER
    for fixed_bpcer in fixed_bpcer_values:
        apcer, threshold = calculate_apcer_at_fixed_bpcer(fpr, tpr, thresholds, fixed_bpcer)
        result = {
            "Dataset": name,
            "Fixed BPCER": f"{fixed_bpcer * 100:.1f}%",
            "APCER": f"{apcer:.3f}",
            "Threshold": f"{threshold:.3f}",
            "Test Accuracy": f"{test_accuracy:.2f}"
        }
        all_results.append(result)

# Create DataFrame
df_results = pd.DataFrame(all_results)

# Display the DataFrame
print(df_results)


3/3 ━━━━━━━━━━━━━━━━━━━━ 26s 7s/step


W0000 00:00:1716300652.606113     126 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


3/3 ━━━━━━━━━━━━━━━━━━━━ 59s 17s/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 61s 18s/step


W0000 00:00:1716300871.664141     125 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


3/3 ━━━━━━━━━━━━━━━━━━━━ 32s 9s/step


W0000 00:00:1716300930.609891     125 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


2/2 ━━━━━━━━━━━━━━━━━━━━ 24s 8s/step
        Dataset Fixed BPCER  APCER Threshold Test Accuracy
0   FaceMorpher        1.0%  0.990     0.786          0.46
1   FaceMorpher       10.0%  0.896     0.635          0.46
2   FaceMorpher       20.0%  0.769     0.560          0.46
3      MIPGAN_I        1.0%  0.996     0.749          0.36
4      MIPGAN_I       10.0%  0.869     0.559          0.36
5      MIPGAN_I       20.0%  0.752     0.474          0.36
6     MIPGAN_II        1.0%  0.979     0.694          0.37
7     MIPGAN_II       10.0%  0.891     0.581          0.37
8     MIPGAN_II       20.0%  0.821     0.532          0.37
9        OpenCV        1.0%  0.997     0.696          0.27
10       OpenCV       10.0%  0.862     0.464          0.27
11       OpenCV       20.0%  0.748     0.344          0.27
12     Webmorph        1.0%  0.986     0.609          0.33
13     Webmorph       10.0%  0.868     0.417          0.33
14     Webmorph       20.0%  0.792     0.312          0.33


W0000 00:00:1716300976.651692     123 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


### **bpcer_at_fixed_apcer**

In [21]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve

def calculate_bpcer_at_fixed_apcer(fpr, tpr, thresholds, fixed_apcer):
    """Calculate the BPCER at a fixed APCER."""
    tpr_target = 1 - fixed_apcer
    closest_tpr_index = np.argmin(np.abs(tpr - tpr_target))
    corresponding_bpcer = fpr[closest_tpr_index]
    corresponding_threshold = thresholds[closest_tpr_index]
    return corresponding_bpcer, corresponding_threshold

# Define datasets, model predictions, and fixed APCER values
datasets = [fm_generator, mg1_generator, mg2_generator, oc_generator, wm_generator]
names = ["FaceMorpher", "MIPGAN_I", "MIPGAN_II", "OpenCV", "Webmorph"]
fixed_apcer_values = [0.01, 0.1, 0.2]
all_results = []

# Iterate over each dataset
for dataset, name in zip(datasets, names):
    # Evaluate the model
    test_loss, test_accuracy = model_ResNet50.evaluate(dataset, steps=dataset.samples // dataset.batch_size)
    
    # Predictions and true labels
    predictions = model_ResNet50.predict(dataset)
    true_labels = dataset.classes
    if predictions.ndim > 1 and predictions.shape[1] > 1:
        predictions = predictions[:, 1]

    # ROC curve metrics
    fpr, tpr, thresholds = roc_curve(true_labels, predictions, pos_label=1)
    
    # Calculate BPCER for each fixed APCER
    for fixed_apcer in fixed_apcer_values:
        bpcer, threshold = calculate_bpcer_at_fixed_apcer(fpr, tpr, thresholds, fixed_apcer)
        result = {
            "Dataset": name,
            "Fixed APCER": f"{fixed_apcer * 100:.1f}%",
            "BPCER": f"{bpcer:.3f}",
#             "Threshold": f"{threshold:.3f}",
            "Test Accuracy": f"{test_accuracy:.2f}"
        }
        all_results.append(result)

# Create DataFrame
df_results = pd.DataFrame(all_results)

# Display the DataFrame
print(df_results)


2/2 ━━━━━━━━━━━━━━━━━━━━ 19s 9s/step - accuracy: 0.4740 - loss: 1.3218
3/3 ━━━━━━━━━━━━━━━━━━━━ 22s 6s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 51s 25s/step - accuracy: 0.3555 - loss: 1.4889
3/3 ━━━━━━━━━━━━━━━━━━━━ 61s 18s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 52s 26s/step - accuracy: 0.4023 - loss: 1.3736
3/3 ━━━━━━━━━━━━━━━━━━━━ 62s 18s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 28s 13s/step - accuracy: 0.2415 - loss: 2.0967
3/3 ━━━━━━━━━━━━━━━━━━━━ 32s 9s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 16s 16s/step - accuracy: 0.3379 - loss: 1.9626
2/2 ━━━━━━━━━━━━━━━━━━━━ 21s 6s/step
        Dataset Fixed APCER  BPCER Test Accuracy
0   FaceMorpher        1.0%  0.995          0.47
1   FaceMorpher       10.0%  0.887          0.47
2   FaceMorpher       20.0%  0.848          0.47
3      MIPGAN_I        1.0%  0.995          0.35
4      MIPGAN_I       10.0%  0.853          0.35
5      MIPGAN_I       20.0%  0.804          0.35
6     MIPGAN_II        1.0%  0.990          0.41
7     MIPGAN_II       10.0%  0.868          0.41
8     MIPGAN_

### **EER**

In [19]:
import numpy as np
from sklearn.metrics import roc_curve
from tensorflow.keras.backend import clear_session

def calculate_eer(true_labels, predictions):
    """Calculate the Equal Error Rate (EER) and the corresponding threshold."""
    # Compute ROC curve
    fpr, tpr, thresholds = roc_curve(true_labels, predictions, pos_label=1)
    
    # Compute FRR (False Rejection Rate)
    frr = 1 - tpr
    
    # Find the EER (Equal Error Rate)
    eer_index = np.argmin(np.abs(fpr - frr))
    eer = fpr[eer_index]
    eer_threshold = thresholds[eer_index]
    
    return eer, eer_threshold

# Define datasets and model predictions
datasets = [fm_generator, mg1_generator, mg2_generator, oc_generator, wm_generator]
names = ["FaceMorpher", "MIPGAN_I", "MIPGAN_II", "OpenCV", "Webmorph"]
all_results = []

# Iterate over each dataset
for dataset, name in zip(datasets, names):
#     clear_session()  # Clear session to free up memory

    # Evaluate the model
    test_loss, test_accuracy = model_ResNet50.evaluate(dataset, steps=dataset.samples // dataset.batch_size)
    
    # Predictions and true labels
    predictions = model_ResNet50.predict(dataset)
    true_labels = dataset.classes
    if predictions.ndim > 1 and predictions.shape[1] > 1:
        predictions = predictions[:, 1]

    # Calculate EER
    eer, eer_threshold = calculate_eer(true_labels, predictions)
    result = {
        "Dataset": name,
        "EER": f"{eer:.3f}",
        "Threshold": f"{eer_threshold:.3f}",
        "Test Accuracy": f"{test_accuracy:.2f}"
    }
    all_results.append(result)

# Create DataFrame
df_results = pd.DataFrame(all_results)

# Display the DataFrame
print(df_results)


2/2 ━━━━━━━━━━━━━━━━━━━━ 18s 9s/step - accuracy: 0.4785 - loss: 1.2803
3/3 ━━━━━━━━━━━━━━━━━━━━ 21s 6s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 50s 25s/step - accuracy: 0.3730 - loss: 1.4713
3/3 ━━━━━━━━━━━━━━━━━━━━ 58s 17s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 50s 25s/step - accuracy: 0.3796 - loss: 1.4686
3/3 ━━━━━━━━━━━━━━━━━━━━ 59s 17s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 26s 13s/step - accuracy: 0.2637 - loss: 2.1007
3/3 ━━━━━━━━━━━━━━━━━━━━ 28s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step - accuracy: 0.3711 - loss: 1.8904
2/2 ━━━━━━━━━━━━━━━━━━━━ 20s 5s/step
       Dataset    EER Threshold Test Accuracy
0  FaceMorpher  0.495     0.273          0.47
1     MIPGAN_I  0.505     0.247          0.38
2    MIPGAN_II  0.490     0.245          0.38
3       OpenCV  0.505     0.109          0.27
4     Webmorph  0.480     0.050          0.37
